In [ ]:
import pandas as pd
patients = pd.read_csv("Patient Matching Data.csv")

In [ ]:
patients.head()

In [ ]:
cleanPatients = patients.drop(["GroupID", "PatientID", "MI"], axis =1)
cleanPatients.head()

In [ ]:
def validateAcc(acc):
    if len(str(acc)) != 11:
        return (float('NaN'))
    else:
        if acc[9] != "-":
            return (float('NaN'))
        else:
            return acc

cleanPatients["Patient Acct #"] = cleanPatients["Patient Acct #"].apply(validateAcc)


In [ ]:
cleanPatients.head(30)

In [ ]:
tokenized_name = cleanPatients['First Name'].apply(lambda x: x.split())

tokenized_name.head()

In [ ]:
from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_name = tokenized_name.apply(lambda x: [ps.stem(i) for i in x])

tokenized_name.head()

In [ ]:
tokenized_name.head(40)

In [ ]:
 def lowerstr(string):
        return(string.lower())
    
cleanPatients["First Name"] = cleanPatients['First Name'].apply(lowerstr)
cleanPatients["Last Name"] = cleanPatients['Last Name'].apply(lowerstr)

cleanPatients.head()

In [ ]:
# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
def convert_state(state):
    if state in us_state_abbrev:
        return us_state_abbrev[state]
    
    return state

cleanPatients["Current State"] = cleanPatients["Current State"].apply(convert_state)
cleanPatients["Previous State"] = cleanPatients["Previous State"].apply(convert_state)

In [ ]:
cleanPatients["Current State"].value_counts()

In [ ]:
cleanPatients["Previous State"].value_counts()

In [ ]:
# Maps letters to phonetic groups
soundex_map = { 
    **dict.fromkeys("BFPV", '1'),
    **dict.fromkeys("CGJKQSXZ", '2'),
    **dict.fromkeys("DT", '3'),
    **dict.fromkeys("L", '4'),
    **dict.fromkeys("MN", '5'),
    **dict.fromkeys("R", '6'),
    **dict.fromkeys("AEIOUHWY", '.'),
}

soundex_map["J"]

In [ ]:
def get_soundex(token):
    """Get the soundex code for the string"""
    token = token.upper()

    # first letter of input is always the first letter of soundex
    soundex = token[0]
    
    for char in token[1:]:
            code = soundex_map[char]
            if code != soundex[-1]:
                soundex += code

    # remove vowels and 'H', 'W' and 'Y' from soundex
    soundex = soundex.replace(".", "")
    
    # trim or pad to make soundex a 4-character code
    soundex = soundex[:4].ljust(4, "0")
        
    return soundex

get_soundex("Maine")

In [ ]:
print(get_soundex("Mervin"))
print(get_soundex("Mikel"))
print(get_soundex("Mikel"))
print(get_soundex("Michael"))
print(get_soundex("Mike"))




In [ ]:
soundex_map["Z"]

In [ ]:

from metaphone import doublemetaphone
from enum import Enum

class Threshold(Enum):
    WEAK = 0
    NORMAL = 1
    STRONG = 2

def double_metaphone(value):
    print(doublemetaphone(value))
    return doublemetaphone(value)

#(Primary Key = Primary Key) = Strongest Match
#(Secondary Key = Primary Key) = Normal Match
#(Primary Key = Secondary Key) = Normal Match
#(Alternate Key = Alternate Key) = Minimal Match
def double_metaphone_compare(tuple1,tuple2,threshold):
    if threshold == Threshold.WEAK:
        if tuple1[1] == tuple2[1]:
            return True
    elif threshold == Threshold.NORMAL:
        if tuple1[0] == tuple2[1] or tuple1[1] == tuple2[0]:
            return True
    else:
        if tuple1[0] == tuple2[0]:
            return True
    return False

In [ ]:
def getMetaphone(string):
    met = double_metaphone(string)
    return met[0]
cleanPatients["metFirst"] = cleanPatients["First Name"].apply(getMetaphone)

In [ ]:
cleanPatients.head()

In [ ]:
fuzzy ratio  